https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb

In [37]:
import gensim, zipfile
from scipy.stats import spearmanr

In [ ]:
f=open("SimLex-999.txt", 'r').readlines()

In [63]:
def add_part(word, part):
    if part == 'A':
        return word+'_ADV'
    if part == 'N':
        return word+'_NOUN'
    if part == 'V':
        return word+'_VERB'

In [70]:
def rank(model_file, part=False):
    with zipfile.ZipFile(model_file, 'r') as archive:
        stream = archive.open('model.bin')
        model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)
        
    #model.distance('apple', 'orange')
    
    not_in_model=[]
    w2v_pairs=[]
    for i in f[1:]:
        ii=i.split('\t')
        first_word=ii[0]
        second_word=ii[1]
        if part:
            first_word=add_part(first_word, ii[2])
            second_word=add_part(second_word, ii[2])
        flag=0
        if first_word not in model:
            not_in_model.append(first_word.split('_')[0])
            flag=1
        if second_word not in model:
            not_in_model.append(second_word.split('_')[0])
            flag=1
        if not flag:
            w2v_pairs.append(model.distance(first_word, second_word))
        #print(first_word, second_word)
    print(len(w2v_pairs), not_in_model)
    
    simlex_pairs=[]
    for i in f[1:]:
        ii=i.split('\t')
        if ii[0] not in not_in_model and ii[1] not in not_in_model:
            simlex_pairs.append(float(ii[3]))
    print(len(simlex_pairs))
    
    return spearmanr(simlex_pairs, w2v_pairs)

## Сначала попробуем без лемматизации

6 	Download 	300 	5 	English Wikipedia Dump of February 2017
	302866 	Gensim Continuous Skipgram 	False

http://vectors.nlpl.eu/repository/20/6.zip

In [65]:
rank('6.zip')

995 ['suds', 'disorganize', 'do', 'do']
995


SpearmanrResult(correlation=-0.39646198627096135, pvalue=8.504530329272487e-39)

Это очень странный результат

## Ok, and what about with lemmatize

5 	Download 	300 	5 	English Wikipedia Dump of February 2017
	273992 	Gensim Continuous Skipgram 	True

http://vectors.nlpl.eu/repository/20/5.zip

In [66]:
rank('5.zip')

997 ['do', 'do']
997


SpearmanrResult(correlation=-0.41016792963353155, pvalue=9.714938327555687e-42)

Вывод в том, что лемматизация не сильно влияет на качество представлений

## Попробуем более новый корпус Википедии, также с лемматизацией

Download 	300 	3 	English Wikipedia Dump of October 2019
	249212 	Gensim Continuous Skipgram 	True

http://vectors.nlpl.eu/repository/20/200.zip

Эта модель содержит части речи

In [71]:
rank('200.zip', True)

894 ['new', 'smart', 'intelligent', 'difficult', 'happy', 'cheerful', 'easy', 'rapid', 'happy', 'glad', 'stupid', 'dumb', 'weird', 'strange', 'awful', 'easy', 'difficult', 'terrible', 'simple', 'smart', 'dumb', 'crazy', 'happy', 'mad', 'large', 'huge', 'new', 'dull', 'rapid', 'dumb', 'foolish', 'wonderful', 'terrific', 'strange', 'odd', 'happy', 'angry', 'broad', 'simple', 'easy', 'apparent', 'obvious', 'inexpensive', 'cheap', 'nice', 'generous', 'weird', 'normal', 'weird', 'odd', 'immoral', 'sad', 'funny', 'wonderful', 'great', 'guilty', 'ashamed', 'beautiful', 'wonderful', 'confident', 'dumb', 'dense', 'large', 'big', 'nice', 'cruel', 'impatient', 'anxious', 'big', 'broad', 'proud', 'unnecessary', 'necessary', 'restless', 'dumb', 'intelligent', 'great', 'difficult', 'simple', 'necessary', 'important', 'terrific', 'mad', 'glad', 'honest', 'guilty', 'easy', 'easy', 'flexible', 'certain', 'essential', 'necessary', 'different', 'normal', 'crucial', 'important', 'harsh', 'cruel', 'childis

SpearmanrResult(correlation=-0.4162699520761669, pvalue=8.844670518902433e-39)

Результат не сильно отличается